In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import numpy as np
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape,Conv2D,MaxPool2D,Flatten,Activation
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
# The path of the saving model check points
save_check_pt = './checkpoints_DANN'

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = cv2.imread(img_dir)
        img = cv2.resize(img,(16,32))
        X.append(img)
    X = np.array(X)
    return X

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = Image.open(img_dir)
#         img = img.convert('L') # conver to grayscale images
#         img = img.resize([16, 32])
#         img_np = np.asarray(img)
#         X.append(img_np.reshape([-1]))
#     X = np.array(X)
#     return X

def get_img_label(path, mode,num_classes=35):
    images_dir, labels, table = load_ccpd(dataset_path[path],mode=mode)
    images = readCcpdImg(images_dir)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    return images,labels

def get_encoder(input_shape=(32,16,3)):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=[3, 3], padding="same", activation='relu',input_shape=input_shape))
    model.add(MaxPool2D(pool_size=[2, 2], strides=2, padding='same'))
    model.add(Conv2D(48, kernel_size=[3, 3], padding="same", activation='relu'))
    model.add(MaxPool2D(pool_size=[2, 2], strides=2, padding='same'))
    model.add(Flatten())
    model.add(Activation('sigmoid'))
    return model

def get_task(input_shape=(1536,)):
    model = Sequential()
    model.add(Dense(1024, activation='relu',input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(34, activation="softmax"))
    return model

def get_discriminator(input_shape=(1536,)):
    model = Sequential()
    model.add(Dense(1024, activation='relu',input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

base_images,base_labels = get_img_label('base', 'all', num_classes=34)
weather_images,weather_labels = get_img_label('weather', 'all', num_classes=34)
weather_images_t,weather_labels_t = get_img_label('weather', 'train', num_classes=34)
challenge_images,challenge_labels = get_img_label('challenge', 'all', num_classes=34)
challenge_images_t,challenge_labels_t = get_img_label('challenge', 'train', num_classes=34)
db_images,db_labels = get_img_label('db', 'all', num_classes=34)
db_images_t,db_labels_t = get_img_label('db', 'train', num_classes=34)
fn_images,fn_labels = get_img_label('fn', 'all', num_classes=34)
fn_images_t,fn_labels_t = get_img_label('fn', 'train', num_classes=34)
print(np.shape(base_images))
#model = get_encoder()
#model.build(input_shape=[None, 32, 16, 3])
#model.summary()
print("load data finished!")

(8446, 32, 16, 3)
load data finished!


In [2]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#Dummy inputs to load model
xs=np.zeros((8446, 32, 16, 3))
ys=np.zeros((8446, 34))
xt=np.zeros((762, 32, 16, 3))
yt=np.zeros((762, 34))
weather_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
weather_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
weather_model.load_weights(save_check_pt+ '/weather')
weather_model.score(weather_images, weather_labels)
weather_hat = weather_model.predict(weather_images)
weather_hat = np.argmax(weather_hat,axis=1)
weather_labels = np.argmax(weather_labels,axis=1)
print(classification_report(weather_labels, weather_hat))

1/1 [==============================] - 0s 446ms/step - loss: 0.2166 - acc: 0.9625
              precision    recall  f1-score   support

           0       0.99      0.94      0.96       248
           1       0.90      0.96      0.93       137
           2       0.99      0.97      0.98       231
           3       0.97      0.99      0.98       172
           4       0.80      1.00      0.89        78
           5       0.98      0.98      0.98       233
           6       0.99      0.97      0.98       236
           7       0.97      0.94      0.96       213
           8       0.97      0.99      0.98       257
           9       0.98      0.99      0.98       224
          10       1.00      0.97      0.98       748
          11       0.85      0.90      0.87        50
          12       1.00      0.96      0.98        49
          13       0.98      0.90      0.93        48
          14       0.77      0.96      0.86        28
          15       0.90      0.95      0.93        40

In [3]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#Dummy inputs to load model
xs=np.zeros((8446, 32, 16, 3))
ys=np.zeros((8446, 34))
xt=np.zeros((109, 32, 16, 3))
yt=np.zeros((109, 34))
challenge_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
challenge_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
challenge_model.load_weights(save_check_pt+ '/challenge')
challenge_model.score(challenge_images, challenge_labels)
challenge_hat = challenge_model.predict(challenge_images)
challenge_hat = np.argmax(challenge_hat,axis=1)
challenge_labels = np.argmax(challenge_labels,axis=1)
print(classification_report(challenge_labels, challenge_hat))

1/1 [==============================] - 0s 172ms/step - loss: 0.8332 - acc: 0.8686
              precision    recall  f1-score   support

           0       0.92      0.90      0.91        39
           1       0.69      0.63      0.66        38
           2       0.94      0.97      0.95        30
           3       0.94      1.00      0.97        16
           4       0.75      0.50      0.60         6
           5       0.85      0.97      0.91        35
           6       0.97      0.89      0.93        37
           7       0.88      0.79      0.83        28
           8       1.00      0.85      0.92        34
           9       1.00      0.74      0.85        42
          10       0.89      0.97      0.93        95
          11       1.00      0.89      0.94         9
          12       0.83      0.71      0.77         7
          13       0.88      0.78      0.82         9
          14       0.80      0.67      0.73         6
          15       0.64      0.88      0.74         8

In [4]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#Dummy inputs to load model
xs=np.zeros((8446, 32, 16, 3))
ys=np.zeros((8446, 34))
xt=np.zeros((373, 32, 16, 3))
yt=np.zeros((373, 34))
db_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
db_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
db_model.load_weights(save_check_pt+ '/db')
db_model.score(db_images, db_labels)
db_hat = db_model.predict(db_images)
db_hat = np.argmax(db_hat,axis=1)
db_labels = np.argmax(db_labels,axis=1)
print(classification_report(db_labels, db_hat))

1/1 [==============================] - 0s 281ms/step - loss: 0.5830 - acc: 0.8866
              precision    recall  f1-score   support

           0       0.92      0.78      0.85       106
           1       0.62      0.95      0.75        37
           2       0.85      0.98      0.91       123
           3       0.92      0.97      0.94       121
           4       0.73      0.90      0.80        39
           5       0.93      0.91      0.92       122
           6       0.90      0.93      0.91       142
           7       0.87      0.89      0.88        75
           8       0.95      0.90      0.92       138
           9       0.92      0.89      0.91       139
          10       1.00      0.91      0.95       372
          11       0.96      0.86      0.91        28
          12       0.72      0.97      0.83        30
          13       1.00      0.50      0.67        14
          14       0.84      0.73      0.78        22
          15       0.76      0.96      0.85        23

In [5]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#Dummy inputs to load model
xs=np.zeros((8446, 32, 16, 3))
ys=np.zeros((8446, 34))
xt=np.zeros((48, 32, 16, 3))
yt=np.zeros((48, 34))
fn_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
fn_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
fn_model.load_weights(save_check_pt+ '/fn')
fn_model.score(fn_images, fn_labels)
fn_hat = fn_model.predict(fn_images)
fn_hat = np.argmax(fn_hat,axis=1)
fn_labels = np.argmax(fn_labels,axis=1)
print(classification_report(fn_labels, fn_hat))

1/1 [==============================] - 0s 175ms/step - loss: 0.0968 - acc: 0.9753
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        16
           1       1.00      0.90      0.95        10
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00         8
           4       0.80      0.80      0.80         5
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        13
           7       1.00      1.00      1.00        12
           8       1.00      0.93      0.96        14
           9       1.00      1.00      1.00        15
          10       0.98      1.00      0.99        52
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         6
          13       1.00      0.75      0.86         4
          15       1.00      1.00      1.00         2
          16       1.00      1.00      1.00         2